In [96]:
%matplotlib auto

import csv
import numpy as np
import matplotlib.pyplot as plt
import math

Using matplotlib backend: Qt4Agg


In [97]:
def parse_data(data_file):
    new_list = []
    with open(data_file) as f:
        reader = csv.reader(f, delimiter = "\t")
        for row in reader:
            new_list.append(row)
    return new_list

#removes the "NA"s in the data and replaces them with 0s
#def remove_blanks(input_list):
#    new_list = [[w.replace("NA", "0") for w in l] for l in input_list]
#    return new_list

#tests the remove_blanks() function
#def test_remove_blanks():
#    a = [["2", "3", "NA"], ["4", "NA", "NA"]]
#    if remove_blanks(a) == [["2", "3", "0"], ["4", "0", "0"]]:
#        return True
#    else:
#        return False
#test_remove_blanks()
    
#takes the data and plots it on the same plot (after deleting the first 3, non-numerical variables)
def plot_data_together(listX, Y_master_list, title):
    #lenY = int(len(Y_master_list[0]))
    #for i in range(len(Y_master_list)):
    #    del Y_master_list[i][0:3]
    #for i in range(1, len(Y_master_list)):
    #    if (int(len(Y_master_list[i])) != lenY):
    #        print("Y value sets are not of same length")
    #        pass
    for i in range(len(Y_master_list)):
        plt.plot(listX, Y_master_list[i])
    plt.xlabel("Days")
    plt.title(title)
    plt.show()
    
    
#given a list of data sets, plots them on individual graphs
def plot_data_separate(listX, Y_master_list):
    fig = plt.figure(1, figsize = (20, 75))
    length = len(Y_master_list)
    plot_height = math.ceil(length / 2)
    for i in range(1, length):
        plt.subplot(plot_height, 2, (i))
        plt.plot(listX, Y_master_list[i])
    plt.show()
    
    
#splits the master list into two lists, one with leaf data, one with apex data
def split_lists_by_tissue(master_list):
    leaf_list = []
    apex_list = []
    for i in range(0, len(master_list)):
        if master_list[i][1] == "leaf":
            leaf_list.append(master_list[i])
        else:
            apex_list.append(master_list[i])
    return leaf_list, apex_list   
    

#splits the list into two - for 'westar' and 'tapidor' samples    
def split_lists_by_species(input_list):
    tapidor_list = []
    westar_list = []
    for i in range(0, len(input_list)):
        if input_list[i][2] == "tapidor":
            tapidor_list.append(input_list[i])
        else:
            westar_list.append(input_list[i])
    
    return tapidor_list, westar_list
    
    
#delete the "NA"s in the tapidor sample 
def clean_data(input_list):
    if input_list[0][2] == "tapidor": 
    
        for i in range(len(input_list)):
            del input_list[i][8]
    
    elif input_list[0][2] == "westar":
        for i in range(len(input_list)):
            del input_list[i][-1]
                
        if input_list[0][1] == "leaf":
        
            for i in range(len(input_list)):
                del input_list[i][8]    
        
    return input_list
        
    
#removes all data with a 0 value
def remove_zero_data(input_list):
    for i in range((len(input_list) - 1), -1, -1):
        if "0" in input_list[i]:
            del input_list[i]
    return input_list      
       

#finds the maximum value in each sublist    
def max_and_min(input_list):
    maximums = []
    minimums = []
    for i in range(len(input_list)):
        t = 0
        for j in range(3, len(input_list[i])):
            if float(input_list[i][j]) > float(t):
                t = input_list[i][j]
        maximums.append(t)
    for i in range(len(input_list)):
        t = maximums[i]
        for j in range(3, len(input_list[i])):
            if float(input_list[i][j]) < float(t):
                t = input_list[i][j]
        minimums.append(t)
    return maximums, minimums   


#normalises the inputted value
def normalise(num, _max, _min):
    return (float(num) - float(_min))/(float(_max) - float(_min))


#given a normalsied list of data, returns those those with their maximum at a particular day and removes the non-numerical data
def select_max_at_n(input_list, max_day):
    if max_day == 22:
        n = 0
    elif max_day == 43:
        n = 1
    elif max_day == 64:
        n = 2
    elif max_day == 65:
        n = 3
    elif max_day == 67:
        n = 4
    elif max_day == 69:
        if "westar" in input_list[0] and "apex" in input_list[0]:
            n = 5
        else:
            print("The selected day is not available for that data")
            pass
    elif max_day == 72:
        if "westar" in input_list[0] and "apex" in input_list[0]:
            print(True)
            n = 6
        else:
            n = 5
    elif max_day == 83:
        if "tapidor" in input_list[0]:
            n = 6
        else:
            print("The selected day is not available for that data")
    new_list = [input_list[i] for i in range (len(input_list)) if input_list[i][n] == 1.0]
    #new_list = [[input_list[i][j] for j in range(3, len(input_list[i]))] for i in range(len(input_list)) if input_list[i][n] == 1]
    
    return new_list



    

master_days = [22, 43, 64, 65, 67, 69, 72, 83]

tapidor_days = [22, 43, 64, 65, 67, 72, 83]
westar_apex_days = [22, 43, 64, 65, 67, 69, 72]
westar_leaf_days = [22, 43, 64, 65, 67, 72]


#preparing gene data
gene_data = parse_data("C:\\Users\\browndo\\Documents\\brassica_flowering_expression.tsv")
#gene_data = remove_blanks(gene_data)
del gene_data[0] #removes the 'header' with the column titles etc.

master_leaf_list, master_apex_list = split_lists_by_tissue(gene_data)

tapidor_leaf, westar_leaf = split_lists_by_species(master_leaf_list)
tapidor_apex, westar_apex = split_lists_by_species(master_apex_list)


tapidor_leaf = remove_zero_data(clean_data(tapidor_leaf))
tapidor_apex = remove_zero_data(clean_data(tapidor_apex))
westar_leaf = remove_zero_data(clean_data(westar_leaf))
westar_apex = remove_zero_data(clean_data(westar_apex))


maxes_t_l, mins_t_l = max_and_min(tapidor_leaf)
maxes_t_a, mins_t_a = max_and_min(tapidor_apex)
maxes_w_l, mins_w_l = max_and_min(westar_leaf)
maxes_w_a, mins_w_a = max_and_min(westar_apex)


normalised_t_l = [[normalise(tapidor_leaf[i][j], maxes_t_l[i], mins_t_l[i]) for j in range(3, len(tapidor_leaf[i]))] for i in range(len(tapidor_leaf))]
normalised_t_a = [[normalise(tapidor_apex[i][j], maxes_t_a[i], mins_t_a[i]) for j in range(3, len(tapidor_apex[i]))] for i in range(len(tapidor_apex))]
normalised_w_l = [[normalise(westar_leaf[i][j], maxes_w_l[i], mins_w_l[i]) for j in range(3, len(westar_leaf[i]))] for i in range(len(westar_leaf))]
normalised_w_a = [[normalise(westar_apex[i][j], maxes_w_a[i], mins_w_a[i]) for j in range(3, len(westar_apex[i]))] for i in range(len(westar_apex))]




def plot_normalised_data():
    plot_data_together(tapidor_days, normalised_t_l, "Normalised data from Tapidor leaf")
    plot_data_together(tapidor_days, normalised_t_a, "Normalised data from Tapidor apex")
    plot_data_together(westar_leaf_days, normalised_w_l, "Normalised data from Westar leaf")
    plot_data_together(westar_apex_days, normalised_w_a, "Normalised from Westar apex")
    
    
def plot_normalised_data_max(max_day):
    normalised_t_l_n = select_max_at_n(normalised_t_l, max_day)
    normalised_t_a_n = select_max_at_n(normalised_t_a, max_day)
    normalised_w_l_n = select_max_at_n(normalised_w_l, max_day)
    normalised_w_a_n = select_max_at_n(normalised_w_a, max_day)
    plot_data_together(tapidor_days, normalised_t_l_n, ("Normalised data from Tapidor leaf with max at %s days") % max_day)
    plot_data_together(tapidor_days, normalised_t_a_n, ("Normalised data from Tapidor apex with max at %s days") % max_day)
    plot_data_together(westar_leaf_days, normalised_w_l_n, ("Normalised data from Westar leaf with max at %s days") % max_day) 
    plot_data_together(westar_apex_days, normalised_w_a_n, ("Normalised from Westar apex with max at %s days")% max_day)
    

#plot_normalised_data()
#for i in [22, 43, 64, 65, 67]:
plot_normalised_data_max(65)




In [92]:
def plot_particular_gene(gene):
    new_list = [gene_data[i] for i in range(len(gene_data)) if gene_data[i][0] == str(gene)]
    maxes, mins = max_and_min(new_list)
    normalised_new_list = [[normalise(new_list[i][j], maxes[i], mins[i]) for j in range(3, len(new_list[i]))] for i in range(len(new_list))]
    
    return normalised_new_list
plot_particular_gene("XLOC_000031")

[[0.33067914600358533,
  0.0,
  0.4109566715559592,
  0.655537705757724,
  1.0,
  0.6968405857838003,
  0.6635235500919653],
 [0.0,
  0.618045387994143,
  1.0,
  0.510935212298682,
  0.7516928989751099,
  0.0177068081991217,
  0.11973828696925377],
 [1.0,
  0.3848483049338004,
  0.11187140058184437,
  0.3555037701122126,
  0.5747936828355996,
  0.020602030517128746,
  0.0],
 [0.5881053441556834,
  1.0,
  0.06605108347605924,
  0.09263534142527341,
  0.08978082259786013,
  0.0]]